# Real-Time Twitter Data Ingestion Using Delta Live Tables (DLT) Streaming Pipeline

#Flow Diagram Of Project

![](/Volumes/data_gov/data_gov_test/image/tweitter_fow.jpg)

### Connect The Tweeter API and extract the Data and Ingest into DBFS ( Databricks File System)